# GHB

In [ ]:
y_num=0

nwhead_fall = kriged_fall[:,0,:]
sehead_fall = kriged_fall[:,-1,:]


# Set kriged water table elevations that are above land surface to land surface minus 15 ft (based on historical levels)
# nwhead_fall = np.where(nwhead_fall>dem_data[0,:], dem_data[0,:] - 15*0.3048, nwhead_fall)
# sehead_fall = np.where(sehead_fall>dem_data[-1,:], dem_data[-1,:]- 15*0.3048, sehead_fall)
dem_offset = 15*0.3048
# dem_offset = 0

kriged_NW = np.where(kriged_NW>dem_data[0,:], dem_data[0,:]- dem_offset, kriged_NW)
kriged_SE = np.where(kriged_SE>dem_data[-1,:], dem_data[-1,:]- dem_offset, kriged_SE)

# calculate the average depth to water table for the spring and from 2013-2018 for the northwest and southeast boundary
avg_nw = np.nanmean(kriged_NW,axis=0)
avg_se = np.nanmean(kriged_SE,axis=0)


nw_lay = get_layer_from_elev(avg_nw, botm[:,0,:], m.dis.nlay)
se_lay = get_layer_from_elev(avg_se, botm[:,-1,:], m.dis.nlay)

# RCH

In [ ]:
import h5py
uzf_dir = join(gwfm_dir, 'UZF_data')
nrow_p, ncol_p = 100,230


In [ ]:
def load_climate(strt_date, end_date):
    nper_tr = (end_date-strt_date).days+1
    ETc = np.zeros((nper_tr,nrow_p,ncol_p))
    agETc = np.zeros((nper_tr,nrow_p,ncol_p))
    rain = np.zeros((nper_tr,nrow_p,ncol_p))
    non_dev_rain = np.zeros((nper_tr,nrow_p,ncol_p))
    ag_rch = np.zeros((nper_tr,nrow_p,ncol_p))

    ETc_count = 0
    for y in np.arange(strt_date.year, end_date.year+1):
        # set start and end date for range for the year to be iterated over
        yr_strt = pd.to_datetime(str(y)+'-01-01')
        yr_end = pd.to_datetime(str(y)+'-12-31')
        # get the length of the date range needed for that year
        yearlen = len(pd.date_range(yr_strt, yr_end))
        if yr_strt < strt_date:
            yr_strt = strt_date
        if yr_end > end_date:
            yr_end = end_date

        # read in text file of all of the ETc data for each year in array format    
#         ET_year = np.loadtxt(gwfm_dir+'/UZF_data/ETa_all_txt_arrays/ETa_array_'+str(y)+'.tsv', delimiter = '\t')
#         rain_year = np.loadtxt(gwfm_dir+'/UZF_data/Rain_all_txt_arrays/Rain_array_'+str(y)+'.tsv', delimiter = '\t')
        # correct the shape of the text file from 2D to 3D
#         revertETc = np.reshape(ET_year, (yearlen, nrow_p, ncol_p))
#         revertrain = np.reshape(rain_year, (yearlen, nrow_p, ncol_p))
        all_ag_arr = np.loadtxt( gwfm_dir+'/WEL_data/simple_landuse_arrays/ag_land_'+str(y)+'.tsv',  delimiter='\t')
        dev_arr = np.loadtxt( gwfm_dir+'/WEL_data/simple_landuse_arrays/developed_land_'+str(y)+'.tsv',  delimiter='\t')
        non_dev_arr = ~dev_arr.astype(bool) # flip array to have non-developed land array for recharge estimates
        # estimate floodplain and Ag recharge
        ag_flood_arr = np.loadtxt(gwfm_dir+'/WEL_data/simple_landuse_arrays/ag_flood_land_'+str(y)+'.tsv', delimiter='\t')
        # aggregate floodplain array over years
    #     ag_flood_arr = np.where(ag_flood_y>0,1,0)

        revertETc = f_clim['evapotranspiration'][str(y)][:]
        revertrain = f_clim['precipitation'][str(y)][:]
        # filter the 3D array based on the desired date range
        filtered_date_ETc = revertETc[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
        filtered_date_rain = revertrain[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
        # get the length of the date range needed for that year
        yr_len = len(pd.date_range(yr_strt, yr_end))
#         print(y, yr_len, ETc[ETc_count:ETc_count+yr_len,:,:].shape, filtered_date_ETc.shape)
        # add the data to the ETc array for the whole model time period
        ETc[ETc_count:ETc_count+yr_len,:,:] = filtered_date_ETc
        agETc[ETc_count:ETc_count+yr_len,:,:] = filtered_date_ETc*all_ag_arr
        rain[ETc_count:ETc_count+yr_len,:,:] = filtered_date_rain
        non_dev_rain[ETc_count:ETc_count+yr_len,:,:] = filtered_date_rain*non_dev_arr
        # extra recharge for alfalfa irrigation to offset lowered water levels
        ag_rch[ETc_count:ETc_count+yr_len,:,:] = filtered_date_ETc*ag_flood_arr
        ETc_count += yr_len
    return(ETc, agETc, rain, non_dev_rain, ag_rch)

In [ ]:
nyrs = end_date.year-strt_date.year
natl_flood_arr = np.zeros((nyrs, nrow,ncol))
for yn, y in enumerate(np.arange(strt_date.year+1, end_date.year+1)):
    # estimate floodplain and Ag recharge
    natl_flood_y = np.loadtxt(gwfm_dir+'/WEL_data/simple_landuse_arrays/natl_flood_land_'+str(y)+'.tsv', delimiter='\t')
    # aggregate floodplain array over years
    natl_flood_arr[yn, natl_flood_y>0] = 1
# aggregate, where vegetation exists in all years
natl_flood_arr = natl_flood_arr.sum(axis=0) >= nyrs

In [ ]:

f_clim = h5py.File(join(uzf_dir, "processed_climate_data.hdf5"), "r")

# f_lu = h5py.File(join(uzf_dir, "simple_land_use_data.hdf5"), "r")


In [ ]:
# load climate data for the model period
old_ETc, old_agETc, rain, non_dev_rain, ag_rch = load_climate(strt_date, end_date)

In [ ]:
ss_ndays = (strt_date - ss_strt).days + 1
# load the recent years data to formulate a steady state period
ss_ETc, ss_agETc, ss_rain, ss_non_dev_rain, ss_ag_rch = load_climate(ss_strt, strt_date)

## Functions to augment recharge
It is expected that alfalfa fields using flood irrigation lead to some recharge and that designated wetland vegetation would experience more flooding. The code below was designed to apply increased recharged. After updating crop mapping this is no longer suited as pumping was greatly reduced and recharge became the issue with overestimation, so I made a basic soil budget model that in a way allows for zones of higher and lower recharge based on soil type.

In [ ]:
# if len(ag_rch.shape) >2:
ag_rch_ss = ag_rch.mean(axis=0)

np.savetxt(model_ws+'/input_data/ag_rch_input.csv', np.reshape(ag_rch, ((nper_tr)*nrow,ncol)))

# np.savetxt(model_ws+'/input_data/ag_rch_input_ss.csv', ag_rch_ss)
# determine ag recharge by loss factor
ag_rch_scaled = ag_rch* scaling_factors.ag_rch_efficiency # assume 5% or 10% irrigation losses for now

In [ ]:
# copy over files from steady state input run
# import shutil, os
# files = ['base_rch_input_ss.csv', 'wel_input_ss.csv', 'ag_rch_input_ss.csv']
# # np.savetxt(model_ws+'/input_data/base_rch_input_ss.csv', net_inf_ss)
# ss_model_input = loadpth+'historical_SS/input_data/'

# for f in files:
#     f = ss_model_input + f
#     shutil.copy(f, m.model_ws+'/input_data')

In [ ]:
np.savetxt(model_ws+'/input_data/natl_flood_arr.csv',natl_flood_arr)
inflow.to_csv(model_ws+'/input_data/inflow.csv', index=True)

# flood stage is 12 ft which corresponds to about 10,000 cfs so when flow is above 10,000 cfs
# 10000 cfs cutoff means it only floods in 2017, lets try 3,000 cfs floodplain connection limit
# 3,000 cfs gives 127 days which is 30 days per year
fp_days = (inflow[inflow.flow_cfs>scaling_factors.fp_active_flow].index - strt_date).days +1 # to account for steady state

fp_rch = np.zeros((nper_tr,nrow,ncol)) # floodplain recharge
# assume constant rate of 1 ft/day for days when flooding happens
fp_rch[fp_days,:,:] = np.ones((nrow,ncol))*natl_flood_arr*0.3048 
# reduce flooding by vka in layer 1
fp_rch = np.where(fp_rch > vka[0,:,:]*natl_flood_arr.astype(bool), vka[0,:,:]*natl_flood_arr.astype(bool), fp_rch)

# any locations more than 10,000 meters from Mokelumne Confluence are less likely to be floodplain inundation
fp_rch[:,:,scaling_factors.fp_column_extent.astype(int):]=0
fp_rch_ss = fp_rch.mean(axis=0)
np.savetxt(model_ws+'/input_data/fp_rch_input_ss.csv', fp_rch_ss)


# UZF

In [ ]:
# required to do this method to insure correct formatting and then change parameters in code below, these are 
# mostly default values
# uzf = flopy.modflow.ModflowUzf1(model = m,
#                                 nuztop=3, iuzfopt=1, irunflg=0, ietflg=1,
#                                 ipakcb=55, # unformatted output file of  ground-water recharge, ET, and ground-water discharge to land surface rates
#                                 iuzfcb2=61,# binary output of recharge and groundwater discharge
#                                 specifythtr = True,
#                                 specifythti = False,
#                                 specifysurfk = True,
#                                 unitnumber=19)

In [ ]:
# nuztop define which cell in column that recharge and discharge is simulated
# 1 = recharge/discharge for top model layer only, 2 = rch/dis to layer specified in iuzfbnd
# 3 = rch/dis to highest active cell in each column, top layer is iuzfbnd, const head node prevents deeper perc
# uzf.nuztop = 3
# # iuzfopt (int) 0 = no unsat zone routing, 1 = kvert specified by UZF in VKS, 2 = kvert taken from LPF or BCF input
# uzf.iuzfopt = 1
# # irunflg (int), whether gw discharge to land surface will be routed to stream (+1) segments specified by irunbnd or removed
# # from the model and accounted as a loss (0)
# # start with removed for simplicitly and this would most likely occur on the lower river where it is near model bounds
# uzf.irunflg = 0
# # ietflg (int) specifies whether ET will be simulated, 0 = no ET, other = ET
# # Start with no ET to get simple UZF running
# uzf.ietflg = 1
# # nstrail2, number of trailing waves used to define he water content profile following a decrease in the infiltration rate
# # range between 10 and 20 is usually adequate
# uzf.ntrail2 = 10
# # nsets2 (in) number of wave sets used to simulate multiple infiltration periods, should be set to 20 for most problems
# uzf.nsets = 30
# # nuzgag = num of cells that will be specified for printing detailed information
# # uzf.nuzgag = 0
# # surfdep = avg undulation depth in the land surface within a finite-difference cell
# uzf.surfdep = (6/12)*0.3048

In [ ]:
# uzf.vks = vka[0,:,:] 
# the low K units end causing too many waves, and if there are 3d paths then vka would be higher
# uzf.vks = vka.mean()
# # try with very high VKS to allow rapid infiltration to water table while maintaining Soil Water Budget
# # if water is below EXTDP then no ET occurs so high VKS kills ET until GW is available
# # uzf.vks = 10

# # specify surfk to set VKA that rejects infiltration
# uzf.surfk = soilKs_array

In [ ]:
# areal extent in which UZF will be simulated
# iuzfbnd = np.copy(ibound[0,:,:])
# # removes -1 from const head boundary and convert to just active vs inactive cells
# iuzfbnd[iuzfbnd[:,:] == -1] = 0
# uzf.iuzfbnd = iuzfbnd[:,:]
# # if irunflg>0 then specify irunbnd
# # array of (int) to define stream segments that overland runoff from excess infil and gw discharge
# # to land surface will be added
# uzf.irunbnd = iuzfbnd[:,:]


In [ ]:
# uzf.eps = soileps_array
# uzf.eps = 3.5 # default value
# # # Convert from percent into decimal
# # maybe overestimating porositiy by using soil data
# # uzf.thts = soiln_array/100
# # porosity estimated with TPROGs, mostly reduces from soil estimate
# uzf.thts = np.mean(tc.get_tprogs_for_elev(por, top, bot1,tprogs_info),axis=0)

In [ ]:
# if 'upw' in globals():
#     # residual water content is equal to the porosity minus specific yield
#     uzf.thtr = uzf.thts.array - upw.sy.array[0,:,:]
# elif 'lpf' in globals():
#     uzf.thtr = uzf.thts.array - lpf.sy.array[0,:,:]


### For each stress period
For the UZF package, rainfall should be directly applied as runoff and ET will both be calculated internally.  
**Steady state** - is consistenly too high with recharge because subtracting ET and some runoff likely misses the overlap of ET due to soil zone residence time and runoff due to steep slopes as well.

In [ ]:
# have transient recharge start after the 1st spd
# finf_dict = {}
# ETc_dict = {}
# for t in np.arange(time_tr0, nper):
# #     finf_dict[t] = np.ones((nrow,ncol))*rain_in[t-time_tr0]
#     finf_dict[t] = rain_arr[t-time_tr0]*(1- soilCN_arr)

#     ETc_dict[t] = ETc[t-time_tr0]

# # set average recharge for previous period as steady state
# # steady state ET only seems to apply to GW, not the soil zone
# if ss_bool == True:
# #     finf_dict[0] = np.ones((nrow,ncol))*rain_in_ss.mean()
#     finf_dict[0] = finf_ss # about 2/3 of assuming average
#     ETc_dict[0] = ss_ETc.mean(axis=0)

In [ ]:
# infiltration rates (L/T) at land surface
# uzf.finf = finf_dict

# # Potential ET (L/T)
# uzf.pet = ETc_dict

# # ET extinction depths (L), 1.5 avg value
# uzf.extdp = 10 # first 3 to give more access, increase to 10
# # Extinction water content, (THTS-Sy) < extwc < THTS
# uzf.extwc = uzf.thtr.array